In [1]:
import sys

%load_ext autoreload
%autoreload 2

sys.path.append("..")

from src.make_predictions import *
from src.data.process_data import *

import sqlite3
import joblib
import pandas as pd

In [2]:
final_stacked_model = joblib.load('../models/finalized_stacked_model')

prev_season_elo = pd.read_csv("../data/clean/final_elo_ratings_2020.csv")

In [3]:
connection = sqlite3.connect('../data/nba.db')

df = load_current_season_team_data(connection, 2021)



In [4]:
home_team = 'PHI'
away_team = 'LAC'
home_spread = -5
away_spread = -home_spread
home_ml = 1.51
away_ml = 2.7

conn = sqlite3.connect('../data/nba.db')
season = 2021

df = load_current_season_team_data(conn, season)

df = clean_team_data(df)

df = prep_for_aggregation(df)

spreads, moneylines = load_current_season_betting_data(conn, 2021)
clean_moneylines = clean_moneyline_df(df = moneylines)
clean_spreads = clean_spreads_df(df = spreads)

full_df = merge_betting_and_boxscore_data(clean_spreads, clean_moneylines, df)
full_df = create_matchups(full_df)
full_df = get_team_and_opp_ewm(full_df, min_periods=1, alpha=0.1)

full_df = add_percentage_features(full_df)
full_df = add_rest_days_for_model(full_df)

current_elo, full_df = get_current_season_elo_ratings(full_df, prev_season_elo)

matchup = create_matchup_rows_for_model(full_df, home_team, away_team,
                                                home_spread, away_spread, home_ml, away_ml)

print("nulls:", matchup.isnull().sum().sum())
matchup

Progress: 100%|██████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.87it/s]
progress:: 2622it [00:00, 6936.75it/s]


nulls: 0


,RECORD_team_diff,FG2M_team_diff,FG2A_team_diff,FG3M_team_diff,FG3A_team_diff,FTM_team_diff,FTA_team_diff,OREB_team_diff,DREB_team_diff,REB_team_diff,...,AST_RATIO_team_diff,AST_RATIO_opp_diff,TOV_PCT_team_diff,TOV_PCT_opp_diff,team_elo_pred,team_rating_i,opp_rating_i,elo_MOV_pred,SPREAD_team,ML_team
0,-0.028741,-1.471485,-5.259264,-2.161367,-2.253147,2.512718,2.260074,-1.089905,-2.163115,-3.253019,...,-1.79605,-1.822412,2.287318,0.247589,0.731181,1633.149039,1528.322706,6.21,-5.0,1.51
1,0.028741,1.471485,5.259264,2.161367,2.253147,-2.512718,-2.260074,1.089905,2.163115,3.253019,...,1.79605,1.822412,-2.287318,-0.247589,0.268819,1528.322706,1633.149039,-6.21,5.0,2.70


In [5]:
spreads.isnull().sum()

SEASON             0
GM_DATE            0
HOME_TEAM          0
AWAY_TEAM          0
AWAY_SCOREBOARD    0
HOME_SCOREBOARD    0
AWAY_SPREAD        0
HOME_SPREAD        0
dtype: int64

In [6]:
connection = sqlite3.connect('../data/nba.db')


main(connection, 2021, final_stacked_model, prev_season_elo, append=True)

Progress: 100%|██████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.93it/s]
progress:: 2624it [00:00, 7454.55it/s]


,game_date,home_team,away_team,home_spread,home_cover_probs,away_cover_probs,home_cover_avg
0,2022-06-03,GSW,BOS,-4.0,0.490438,0.511213,0.489612


In [7]:
final_stacked_model_sfs = joblib.load('../models/finalized_stacked_model_retrain-21-12-20')

connection = sqlite3.connect('../data/nba.db')


main(connection, 2021, final_stacked_model_sfs, prev_season_elo, append=False)


Progress: 100%|██████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.94it/s]
progress:: 2624it [00:00, 7476.02it/s]


,game_date,home_team,away_team,home_spread,home_cover_probs,away_cover_probs,home_cover_avg
0,2022-06-03,GSW,BOS,-4.0,0.494127,0.519424,0.487352


In [8]:
final_stacked_model_v2 = joblib.load('../models/final_stacked_clf_v2-21-12-20')

connection = sqlite3.connect('../data/nba.db')

main(connection, 2021, final_stacked_model_v2, prev_season_elo, append=False)


Progress: 100%|██████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.93it/s]
progress:: 2624it [00:00, 7476.22it/s]


,game_date,home_team,away_team,home_spread,home_cover_probs,away_cover_probs,home_cover_avg
0,2022-06-03,GSW,BOS,-4.0,0.485299,0.521643,0.481828
